# Dependencies

In [29]:
from dotenv import load_dotenv
load_dotenv()

import os
import sys
import time

# CrewAI
from crewai import Agent, Task, Crew, Process
from textwrap import dedent


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")


In [30]:
import warnings 
warnings.filterwarnings('ignore')

# Agents
Here, we will be building individual agents which will work in sync to research and write a blog. For this, we will be using crewAI agents to create individual agents and combine them to fulfill the common task of researching and writing the blog.

In [31]:
researcher = Agent(
    role = 'researcher',
    goal = 'To research and get accurate resources on the topic : {topic}',
    backstory = """
                    You are a content planner in an organisation
                    where your expertise is searching for accurate and fact checked resources
                    on the given topic : {topic}, allowing people to learn something meaningful.
                    Your work will act as the basis for the writer to write a blog for 
                    our auidence. You can also use the tool below to search the internet for these
                    topics.
    """,
    verbose = True,
    allow_delegation = False
)

So here above, the agents are working on a process of **roleplaying**, where each agent is given a goal and a background. This allows them to perform better when they are playing the role of someone/something specific and have a set target to achieve. 

## Roleplaying
1. The better we give context to the agent about how to perform a task, and what it's background and functioning is suppose to be like, the better it will perform. Think of it in a RAG based manner, when you mention the agent of being a "financial anaylyst", it will scrape the net in a generic way to fetch metrics in it's RAG apporach. 

2. However, if you mention in it's story that you are a FINRA approved financial analyst, it will scrape the net in a more specific way to fetch metrics in it's RAG approach, somehow similar to how a FINRA approved financial analyst would do.

## Focus 
1. As we push in more text or content into the context window of any LLM, it has shown in research that with time, it will loose out on key information and in turn open up opportunities for hallucinations.
2. Since for each task or broad activity of a process, you have made different agents, the overall context is now distributed and hence, each agent has a higher propensity to focus.

## Tools
1. You can hyper specify which tools a given agent should use, so that an agent does not use wrong tools or generate wrong answers. In this way, these agents further build on their aspect of focus and efficient output.

## Cooperation
1. Since all these agents tend to work together, they can simulate the behaviours of feedback and task delegation amongst themselves, something similar to what a human in the loop does with ChatGPT, enabling better output.

## Guradrails
1. In most AI solutions, their remains the case of fuzzy inputs, fuzzy transformations and fuzzy outputs. The AI system (ChatGPT) can convert the output to any format and their is no fixed types we can predict.
2. In such agents, it is preset in frameworks to have guardrails to protect from such things and yield input and output in the way the user specifies.

## Memory
1. There are primarily three kinds of memory system we are provided
   1. Short-term memory 
   2. Long-term memory
   3. Entity memory

2. **Short Term Memory**
   1. Memory during the execution task
   2. Share knowledge, activities and learnings with other agents
   3. Sharing immediate information even before providing "tast-completion" output

3. **Long Term Memory**
   1. Memory is stored after execution of current tasks, usually in a local DB.
   2. Long term memory can be used in any future tasks
   3. With this, we in theory have "self-improving" agents

4. **Entity Memory**
   1. In case of a longer context window, the system may also store the entity memory (such as name, entity, subject and object of the sentence/text)

Together, all these memory types work to work on "self-improving" agents as a function of time.

Similarly, we can create the agents for writer and editor as well.

In [32]:
writer = Agent(
    role = 'writer',
    goal = 'To write a insightful, well referenced opinion piece on the topic  : {topic}',
    backstory = """
                    You are a content writer working on writing a opinionated content piece on 
                    the topic : {topic}. You role is based on the work of a researcher in your team, 
                    who will be giving all the research articles and facts collated. You will be writing 
                    your opinion piece with acknowledging and referencing each factual statement as given by the researcher
                    eventually writing an opinion piece.
    """,
    verbose = True,
    allow_delegation = False
)

In [33]:
editor = Agent(
    role = 'editor',
    goal = 'To edit an opinion piece given by a writter to suit your organisation writing style',
    backstory = """
                    You are an editor working in an organisation which runs their own social media blogging 
                    site for young demographics. You work on editing the written blog in a fun and playful way, 
                    so as to appeal to your readers. However, it is a top priority at your end not to omit
                    any references to factual data done by the writer, as it aids in building authenticity of your 
                    final article.
    """,
    verbose = True,
    allow_delegation = False
)

# Tasks
Now, we will be clearly defining tasks for agent. This will be a specific assignment which has to be completed by the agent. This will help the agent to understand the goal and work towards it effectively.

In [34]:
research = Task(
    description=(
        "1. Prioritize the latest trends, key players, and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline including an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive researched document with an outline, audience analysis, SEO keywords, and resources.",
    agent=researcher,
)

In [35]:
write = Task(
    description = (
        """
            1. Use the content plan to craft a compelling blog post on {topic}.\n
            2. Incorporate SEO keywords naturally.\n
		    3. Sections/Subtitles are properly named in an engaging manner.\n
            4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.\n
            5. Proofread for grammatical errors and alignment with the brand's voice.\n
        """
    ),
    expected_output = "A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent = writer
)

In [36]:
edit = Task(
    description = "Proofread the given blog post for grammatical errors and alignment with the brand's voice.",
    expected_output = "A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent = editor
)

# Crew
Now, we will be creating a team which will work in sync to complete the process of writing a blog by researching it, given the topic. This will help in creating a team of agents which will work together to achieve the common goal. By default, the crew operates sequentially, implying that the agents will work one after the other, and depend on the input from the previous agent.

In [37]:
crew = Crew(
    agents = [researcher,writer,editor],
    tasks = [research,write,edit],
    verbose = True
)

Overriding of current TracerProvider is not allowed


# Execution
Now, we can work on executing the crew to complete the task of writing a blog. 

In [38]:
result = crew.kickoff(inputs={'topic':"OpenAI o3 mini vs OpenAI o1"})

# Agent: researcher
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on OpenAI o3 mini vs OpenAI o1.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: researcher
## Final Answer: 
### OpenAI O3 Mini vs OpenAI O1: A Comprehensive Analysis

#### Target Audience
The target audience for this content includes:
- **AI Enthusiasts:** Individuals who are passionate about artificial intelligence, its applications, and technology advancements.
- **Developers and Data Scientists:** Professionals looking to incorporate OpenAI's models into their projects or understand their functionalities.
- **Tech-Savvy Business Executives:** Decision-makers focused on leveraging AI for business growth and innovation.
- **Educators and Students:** Those interested in learning about AI's emerging tr

Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 1375, in getresponse
    response.begin()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 279, in _read_status
    line = str(self.fp.r



# Agent: editor
## Final Answer: 
```markdown
# OpenAI O3 Mini vs. OpenAI O1: A Playful Peek into the AI Arena

## Introduction

Welcome to the exciting world of artificial intelligence, where innovations are constantly reshaping our reality! OpenAI is at the forefront of this revolution, offering dazzling models that cater to various applications. Every tech-savvy business, developer, and curious researcher is looking to harness the power of AI, which makes it super important to understand the subtle nuances between different models. Today, we're diving into a head-to-head comparison of two heavyweights in OpenAI's lineup – the O3 Mini and the O1 models. Buckle up!

The O3 Mini and O1 aren't just fancy names; they're transformational tools that bolster OpenAI’s promise of high-performance solutions across natural language processing, computer vision, and beyond. In this blog, we'll uncover their unique features, strengths, and ideal scenarios for use. Our mission? To guide tech enth

In [40]:
from IPython.display import Markdown,display
display(Markdown(result.raw))

```markdown
# OpenAI O3 Mini vs. OpenAI O1: A Playful Peek into the AI Arena

## Introduction

Welcome to the exciting world of artificial intelligence, where innovations are constantly reshaping our reality! OpenAI is at the forefront of this revolution, offering dazzling models that cater to various applications. Every tech-savvy business, developer, and curious researcher is looking to harness the power of AI, which makes it super important to understand the subtle nuances between different models. Today, we're diving into a head-to-head comparison of two heavyweights in OpenAI's lineup – the O3 Mini and the O1 models. Buckle up!

The O3 Mini and O1 aren't just fancy names; they're transformational tools that bolster OpenAI’s promise of high-performance solutions across natural language processing, computer vision, and beyond. In this blog, we'll uncover their unique features, strengths, and ideal scenarios for use. Our mission? To guide tech enthusiasts, daring developers, and business trailblazers through the wild world of AI to make savvy, well-informed choices that fit their needs like a glove!

## Overview of OpenAI Models

OpenAI models are like the sharpest tools in the AI shed, built with sophisticated algorithms that imitate human thought processes through machine learning magic. From natural language processing (NLP) to computer vision, the innovative work of OpenAI is stirring up industry-wide transformation. Through the years, we've seen these models evolve – each iteration craftily improving in performance, scalability, and usability. The journey that leads to the O3 Mini and O1 reflects a forward-thinking approach to tackle the ever-changing needs in the AI landscape, directly targeting user and business demands.

It’s absolutely vital to grasp the key differences between these models if you want to unlock the full potential of AI for your projects. Meet the O3 Mini, a champion of efficiency with its lightweight framework, designed to steal the spotlight for developers seeking a smooth ride. Meanwhile, the O1 model is the heavyweight contender, perfect for users craving that extra oomph in computational prowess. Knowing what’s on the table with each model can utterly transform how effective your applications are, whether they're for personal use or commercial gain.

## OpenAI O3 Mini

Say hello to the OpenAI O3 Mini, a sprightly little model crafted to bring you the perfect balance between performance and resource consumption. With its featherweight footprint, the O3 Mini dazzles with speedy response times and enhanced task-processing efficiency. It's like the perfect sidekick for any developer looking to fuse AI smoothly into existing systems without tearing everything apart!

But let’s keep it real; the O3 Mini has some limitations, too. When faced with grand projects requiring an avalanche of computational power, it might not shine as brightly as its heftier counterparts. However, where it truly excels is in applications that prioritize quick and manageable outputs. Think chatbots, agile web applications, and light NLP tasks – this model proves itself to be a superstar for anyone eager to deploy AI solutions swiftly and economically!

## OpenAI O1

On the flip side, let’s roll out the red carpet for the OpenAI O1 model! This powerhouse is ready to tackle demanding tasks while delivering top-notch outputs that leave the competition in the dust. It's packed with robust features, including access to extensive training datasets, impressive scalability, and multifaceted application capabilities. It’s like the Swiss Army knife of AI, perfect for projects that are all about heavy lifting — think large-scale NLP applications, deep data analysis, and complex machine learning adventures.

Yet, there’s a twist in the tale! The O1's hunger for resources might mean shelling out some serious dough for the infrastructure to support effective deployment. But fear not! Countless success stories spill the beans on O1's prowess in the real world, like streamlining automated translations for global businesses and powering advanced recommendation systems. If organizations are ready to invest their sweat and treasure into robust solutions, O1 is a contender not to be overlooked!

## Comparative Analysis

When we pit these two titans against one another, we see a colorful tapestry of strengths and weaknesses. The O3 Mini is the efficiency whiz, firing on all cylinders for smaller tasks while being kind on budgets. However, its small size might limit its performance in heavyweight scenarios, which is where the O1's might shines through! O1 is the go-to for those heavyweight projects needing high computational power, but it comes with its own baggage of higher costs and resource demands.

For organizations contemplating their next big step, the choice between O3 Mini and O1 boils down to specific project needs. Smaller ventures or startups may find their happy place in the efficiency of O3 Mini, while larger enterprises or data-driven powerhouses might be drawn to the expansive capabilities of O1. Understanding the implications of this choice is key for optimizing AI’s impact on performance and return on investment. 

## Future Trends and Developments

Looking ahead, the world of AI is awash with exciting possibilities, and both O3 Mini and O1 are set for thrilling developments. OpenAI is on a mission to keep refining its models, fueled by user feedback and market insights. Expect updates that bump up functionality, performance, and user-friendliness. Many predict that future iterations will come loaded with intuitive interfaces and smarter machine learning capabilities, shaking up how AI is integrated across different sectors!

With the meteoric rise of AI in business solutions, both O3 Mini and O1 users can tap into increased efficiency and sharpen their competitive edge. Staying in the know about these developments and market trends is crucial for stakeholders eager to make the most of OpenAI's advanced offerings!

## Conclusion

Choosing the right AI model is a monumental decision for developers, businesses, and educators. The OpenAI O3 Mini and O1 models are tailored with distinct features and perks designed for various needs. Grasping these differences is key to maximizing your AI application potential. As the industry shifts and evolves, continuing to learn about AI technologies will only strengthen your knowledge and ability to adapt! 

We’d love to hear from you! Share your experiences with OpenAI models and jump into discussions about the value these robots can bring to your field. Don't forget to stay updated on OpenAI's latest progress and trends, as you harness the incredible power of AI!
```

# Conclusion
With this, we can see how we can create a team of agents which can work together to achieve a common goal. This acts an introduction to the concept of roleplaying and how it can be used to create a team of agents which can work together to achieve a common goal.